In [58]:
from keras.applications import MobileNet

In [59]:
img_rows, img_cols=224,224

MobileNet = MobileNet(weights = 'imagenet', include_top = False , input_shape = (img_rows, img_cols,3))

for layer in MobileNet.layers:
    layer.trainable = False
    

In [60]:
def lw(bottom_model , num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation='relu')(top_model)
    top_model = Dense(1024, activation='relu')(top_model)
    top_model = Dense(512, activation='relu')(top_model)
    top_model = Dense(num_classes, activation='softmax')(top_model)
    return top_model
    

In [61]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 5 

FC_Head = lw(MobileNet, num_classes)
model=Model(input = MobileNet.input , output = FC_Head)

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 32)      0         
__________

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.


In [62]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'face/train'
validation_data_dir = 'face/val'

train_datagen = ImageDataGenerator(rescale = 1./255, rotation_range=45, width_shift_range=0.3, height_shift_range=0.3, horizontal_flip=True, fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_rows, img_cols), batch_size=batch_size,class_mode='categorical')
validation_generator = validation_datagen.flow_from_directory(validation_data_dir,target_size=(img_rows, img_cols),batch_size=batch_size,class_mode='categorical')


Found 93 images belonging to 5 classes.
Found 25 images belonging to 5 classes.


In [63]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("face_recog.h5", monitor="val_loss", mode="min",save_best_only=True,verbose=1)

earlystop = EarlyStopping(monitor='val_loss', min_delta=0 , patience=3, verbose=1)

callbacks = [earlystop , checkpoint]

model.compile(loss = 'categorical_crossentropy', optimizer = RMSprop(lr = 0.001), metrics=['accuracy'])

nb_train_samples = 92
nb_validation_samples = 25

epochs = 5
batch_size = 16

history = model.fit_generator(train_generator, steps_per_epoch = nb_train_samples//batch_size, epochs=epochs , callbacks=callbacks, validation_data=validation_generator, validation_steps = nb_validation_samples//batch_size)


Epoch 1/5
5/5 [==============================] - 33s - loss: 9.0266 - acc: 0.2682 - val_loss: 12.8945 - val_acc: 0.2000
Epoch 2/5
5/5 [==============================] - 27s - loss: 12.8041 - acc: 0.2056 - val_loss: 12.8945 - val_acc: 0.2000
Epoch 3/5
5/5 [==============================] - 26s - loss: 12.1436 - acc: 0.2466 - val_loss: 12.8945 - val_acc: 0.2000
Epoch 4/5
5/5 [==============================] - 25s - loss: 12.0063 - acc: 0.2551 - val_loss: 12.8945 - val_acc: 0.2000
Epoch 5/5
5/5 [==============================] - 25s - loss: 12.8881 - acc: 0.2004 - val_loss: 12.8945 - val_acc: 0.2000


In [66]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

celeb_dict = {"[0]": "ben_afflek ", 
                      "[1]": "elton_john",
                      "[2]": "jerry_seinfeld",
                      "[3]": "madonna",
                      "[4]": "mindy_kaling ",
                     }

celeb_dict_n = {"ben_afflek": "ben_afflek ", 
                      "elton_john": "elton_john",
                      "jerry_seinfeld": "jerry_seinfeld",
                      "madonna": "madonna",
                      "mindy_kaling": "mindy_kaling ",
                     }

def draw_test(name, pred, im):
    celeb =celeb_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, monkey, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " +celeb_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("face/val/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    #res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    #draw_test("Prediction", res, input_original) 
    #cv2.waitKey(0)

cv2.destroyAllWindows()


Class - madonna
Class - ben_afflek 
Class - madonna
Class - jerry_seinfeld
Class - mindy_kaling 
Class - jerry_seinfeld
Class - mindy_kaling 
Class - madonna
Class - jerry_seinfeld
Class - mindy_kaling 
